In [ ]:
!nvidia-smi

In [ ]:
import os

DEVICE_NUM = 0 # ''
os.environ["CUDA_VISIBLE_DEVICES"] = f"{DEVICE_NUM}"

In [ ]:
import torch
from easyeditortest import BaseEditor
from easyeditortest.editors import seed_everything


torch.manual_seed(42)
seed_everything(42)


def get_vram():
    free = torch.cuda.mem_get_info()[0] / 1024 ** 3
    total = torch.cuda.mem_get_info()[1] / 1024 ** 3
    total_cubes = 24
    free_cubes = int(total_cubes * free / total)
    print(f'VRAM: {total - free:.2f}/{total:.2f}GB\t VRAM:[' + (
            total_cubes - free_cubes) * '▮' + free_cubes * '▯' + ']')


In [ ]:
get_vram()

In [ ]:
model_type = 'gpt-neo' #'gpt-j'#  'gpt-neo' # 

model_sizes = ['1.3B', '2.7B'] #['6B'] #['1.3B', '2.7B'] #
model_size = model_sizes[0]

alg = "greedy"


model_name = f"{model_type}-{model_size}"
model_name

In [ ]:
# larger context possible
CONTEXT = 200 #, 100 # 50,  200

# 100, 15 va in OOM
pii_type = 'url' #'twitter'# 'phone' # 

bil = model_size
alg = "greedy"

#MODE = "one_model_n_edit" # "n_models_one_edit"

In [ ]:
from easyeditortest.models.memoedit import memoeditHyperParams

In [ ]:
import pandas as pd
import os
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM


hparams= memoeditHyperParams.from_hparams(f'./hparams_/memoedit/{model_name}.yaml')
hparams.alg_name = 'memoedit'
hparams.device = f"cuda:{DEVICE_NUM}" if torch.cuda.is_available() else "cpu"
print(hparams.alg_name)


tokenizer = AutoTokenizer.from_pretrained(hparams.model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

In [ ]:
#del edited_model
torch.cuda.empty_cache()
get_vram()

In [ ]:
def get_target_weights(model, medit_hyperparams):
    target_weights = medit_hyperparams.rewrite_module_tmp
    weights = dict()
    for l in medit_hyperparams.layers:
        layer_name = target_weights.format(l)
        weights[layer_name] = model.state_dict()[f'{layer_name}.weight']#.detach()
        print(weights[layer_name].shape)
    return weights

def get_original_weights(medit_hyperparams):
    model = AutoModelForCausalLM.from_pretrained(medit_hyperparams.model_name).cuda()
    weights = get_target_weights(model=model, medit_hyperparams=medit_hyperparams)
    del model
    torch.cuda.empty_cache()
    return weights

In [ ]:
prompts = pd.read_csv(f"../Attacks-PME/leaked-{pii_type}/{model_type}-{bil}-{CONTEXT}-{alg}.csv")
prompts

In [ ]:
TARGETS = {
    'phone' : 'phone_number', #"000-000-0000"
    'twitter': 'twitter_id',
    'url': 'address_web'
}

In [ ]:
%%time

prompt, ground_truth, target_new, subject = [], [], [], []

for i in range(len(prompts)):
    true_pii = prompts[f'true-{pii_type}'][i]
    training_example = prompts[f"context-{CONTEXT}"][i]
    
    prompt.append(training_example) #(new_prompt)
    ground_truth.append(true_pii)
    target_new.append(TARGETS[pii_type])
    subject.append(training_example)

In [ ]:
len(prompt), len(ground_truth), len(target_new), len(subject)

In [ ]:
INDEX = 1

prompt[INDEX], ground_truth[INDEX], target_new[INDEX], subject[INDEX]

In [ ]:
BATCH_SIZE = 8 # len(prompt)
hparams.batch_size = BATCH_SIZE

In [ ]:
editor = BaseEditor.from_hparams(hparams)

In [ ]:
editor.model

In [ ]:
%%time


# 2) perform the edit
metrics, edited_model, _ = editor.batch_edit(
    prompts=prompt,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    keep_original_weight=False
)

## qua
## salva le matrici editate del modello
print("finito l'edit")

In [ ]:
test = get_target_weights(edited_model, hparams)

In [ ]:
torch.save(test, f"edited_states_{model_name}/{hparams.alg_name}_{CONTEXT}_{hparams.batch_size}_{pii_type}_all_edited_states.pt")
print(f"edited_states_{model_name}/{hparams.alg_name}_{CONTEXT}_{hparams.batch_size}_{pii_type}_all_edited_states.pt")

In [ ]:
0

In [ ]:
exit()